In [10]:
from queue import PriorityQueue
import pandas as pd
import numpy as np
from collections import defaultdict
import heapq

In [11]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [12]:
class Graph:

    def __init__(self, adj_matrix=None, edge_list=None, directed = False):
        if adj_matrix:
            self.adj_matrix = pd.DataFrame(adj_matrix)
        else:
            self.adj_matrix = None

        self.edge_list = edge_list
        self.adj_list = defaultdict(list)
        self.directed = directed
        self.visited = []

        if not self.adj_list and self.edge_list:
            self.prepare_adj_list()


    def add_edge(self, u, v, w):
        if not self.edge_list:
            self.edge_list = []
        self.edge_list.append([u, v, w])
        self.adj_list[u].append((v, w))
        self.adj_list[v].append((u, w))

    def prepare_adj_list(self):
        for u, v, w in self.edge_list:
            self.adj_list[u].append((v, w))
            if self.directed:
                self.adj_list[v].append(None)
            else:
                self.adj_list[v].append((u,w))

    def prepare_adj_matrix(self):
        self.vertices = sorted(self.adj_list.keys())
        n_vertices = len(self.vertices)
        self.edges = [[None for i in range(n_vertices)] for j in range(n_vertices)]
        for i in range(n_vertices):
            for j in range(n_vertices):
                if i==j:
                    self.edges[i][j] = 0

        self.adj_matrix = pd.DataFrame(self.edges, index=self.vertices, columns=self.vertices)
        # print(self.adj_matrix)

        for k, v in self.adj_list.items():
            for x in v:
                if x:
                    self.adj_matrix.loc[k][x[0]] = int(x[1])
        self.adj_matrix.fillna(np.inf, inplace=True)

        # print(self.adj_matrix)

    def dijkstra(self, start_vertex):
        if self.adj_matrix is None:
            self.prepare_adj_matrix()
        num_vertices = self.adj_matrix.shape[0]
        # vertices = self.adj_matrix.index.values
        # print(vertices)
        print('Adjacency Matrix:')
        display(self.adj_matrix)

        print(f'\nStarting vertex: {start_vertex}')

        D = {v: np.inf for v in self.vertices}
        V = {v: None for v in self.vertices}
        D[start_vertex] = 0

        pq = PriorityQueue()
        pq.put((0, start_vertex))

        shortest_path_list = {}

        while not pq.empty():
            (dist, current_vertex) = pq.get()
            print(f'\nCurrent vertex: {current_vertex}')
            self.visited.append(current_vertex)
            print(f'Visited Vertices: {self.visited}')

            for neighbor in self.vertices:
                if self.adj_matrix.loc[current_vertex][neighbor] != np.inf:
                    distance = self.adj_matrix.loc[current_vertex][neighbor]
                    print(f'\n\nNeighbour: {neighbor}, Distance: {neighbor}')
                    #print(f'\nDistance: {neighbor}')
                    if neighbor not in self.visited:
                        print(f'\nNeighbour {neighbor} not visited')
                        old_cost = D[neighbor]
                        new_cost = D[current_vertex] + distance
                        print(f'\nOld cost: {old_cost}, New cost: {new_cost}')
                        if new_cost < old_cost:
                            print(f'\nNew cost less than old cost, so neighbour {neighbor} added to queue')
                            pq.put((new_cost, neighbor))
                            D[neighbor] = new_cost
                            V[neighbor] = current_vertex
            S = []
            u = current_vertex
            while V[u] != None:
                S.insert(0, u)
                u = V[u]

            S.insert(0, start_vertex)
            shortest_path_list[current_vertex] = S
            print('\n------------------------------------------------------------------')
        #print(np.array([self.vertices, AllPathsList, D]).T)

        path_df = pd.DataFrame(self.vertices, columns=['Vertex'])
        #print(shortest_path_list)

        path_df['Shortest Path'] = path_df['Vertex'].apply(lambda x: shortest_path_list.get(x) if shortest_path_list.get(x)
                                                           else 'No Path Exists' )
        path_df['Shortest Distance'] = path_df['Vertex'].apply(lambda x: D[x])
        print(f'\nThe Optimal Solution or Shortest Path from Source Vertex {start_vertex} is :\n')
        display(path_df)

In [13]:
g = Graph(edge_list=[(0, 1, 4), (0, 2, 7), (1, 2, 11),(1, 3, 20), (3, 4, 5), (3, 5, 6),
                     (2, 3, 3),(2, 4 ,2)])


g.dijkstra(start_vertex=1)

Adjacency Matrix:


,0,1,2,3,4,5
0,0.0,4.0,7.0,inf,inf,inf
1,4.0,0.0,11.0,20.0,inf,inf
2,7.0,11.0,0.0,3.0,2.0,inf
3,inf,20.0,3.0,0.0,5.0,6.0
4,inf,inf,2.0,5.0,0.0,inf
5,inf,inf,inf,6.0,inf,0.0



Starting vertex: 1

Current vertex: 1
Visited Vertices: [1]


Neighbour: 0, Distance: 0

Neighbour 0 not visited

Old cost: inf, New cost: 4.0

New cost less than old cost, so neighbour 0 added to queue


Neighbour: 1, Distance: 1


Neighbour: 2, Distance: 2

Neighbour 2 not visited

Old cost: inf, New cost: 11.0

New cost less than old cost, so neighbour 2 added to queue


Neighbour: 3, Distance: 3

Neighbour 3 not visited

Old cost: inf, New cost: 20.0

New cost less than old cost, so neighbour 3 added to queue

------------------------------------------------------------------

Current vertex: 0
Visited Vertices: [1, 0]


Neighbour: 0, Distance: 0


Neighbour: 1, Distance: 1


Neighbour: 2, Distance: 2

Neighbour 2 not visited

Old cost: 11.0, New cost: 11.0

------------------------------------------------------------------

Current vertex: 2
Visited Vertices: [1, 0, 2]


Neighbour: 0, Distance: 0


Neighbour: 1, Distance: 1


Neighbour: 2, Distance: 2


Neighbour: 3, Distance: 3


,Vertex,Shortest Path,Shortest Distance
0,0,"[1, 0]",4.0
1,1,[1],0.0
2,2,"[1, 2]",11.0
3,3,"[1, 2, 3]",14.0
4,4,"[1, 2, 4]",13.0
5,5,"[1, 2, 3, 5]",20.0
